# Notebook 05. Using RLlib with Ray Serve to deploy a policy into production

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives
In this this notebook, you will learn:

 * [How to deploy a trained policy into production using Ray Serve](#ray_serve)
 * [How to send requests to a deployed policy via HTTP](#ray_serve_requests)

In [1]:
# import required packages

import gym
import numpy as np
import requests
from requests import Request
import tree  # pip install dm_tree

import ray
from ray import serve
from ray.rllib.algorithms.crr import CRRConfig

print(f"gym: {gym.__version__}")
print(f"ray: {ray.__version__}")

# !ale-import-roms --import-from-pkg atari_py.atari_roms

gym: 0.21.0
ray: 3.0.0.dev0


### Using Ray Serve to deploy a trained policy into production <a class="anchor" id="ray_serve"></a>

<img src="images/rllib_and_ray_serve.png" width=800 />

This is a quick demo on how to use our already (offline RL) trained CRR policy and deploy it using Ray Serve.
All you need to run the following code and produce a serve deployment is one of the checkpoints files from the previous notebook. Let's take checkpoint number 

In [2]:
# Call `serve.start()` to get 
serve.start()


@serve.deployment(route_prefix="/serve-deployment")
class ServeModel:
    def __init__(self, config, checkpoint) -> None:
        # Create new algo from scratch.
        self.algo = config.build()
        # Restore state of algo to a already trained one (using a checkpoint).
        self.algo.restore(checkpoint)

    async def __call__(self, request):
        json_input = await request.json()
        # Extract observation from input.
        obs = json_input["observation"]
        # Translate obs back to np.arrays.
        np_obs = np.array(obs)
        action = self.algo.compute_single_action(np_obs, explore=False)
        return {"action": action}


Usage stats collection is enabled by default for nightly wheels. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


2022-08-15 16:19:58,520	INFO worker.py:1481 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267.
(ServeController pid=21320) INFO 2022-08-15 16:20:02,010 controller 21320 http_state.py:123 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SNCAYa:SERVE_PROXY_ACTOR-f32e32c78857a53ea3859781ba7c78d31fe11bd135496dcf1c20fc3f' on node 'f32e32c78857a53ea3859781ba7c78d31fe11bd135496dcf1c20fc3f' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=20076) INFO:     Started server process [20076]


In the following cell, you will be asked to provide the path to an already existing checkpoint file.

We have created Pendulum-v1 CRR checkpoints in the previous notebook and you should by now be able to navigate to one of these checkpoints using your notebook file browser on the left side:

<img src="images/copy_checkpoint_path.png" width=400>

Once you copied the path, you can paste it into the `checkpoint = "..."` code below:

In [3]:
# In order to create a deployment from the tagge class above, simply call its `deploy()`
# method and pass this method the `ServeModel` constructor arguments:

# Create config to use. Same as before, but lighter:
# 1) No evaluation necessary (model only used for inference).
# 2) No `offline_data` settings necessary (model only used for inference).

config = CRRConfig().environment(env="Pendulum-v1")
config.framework("torch")

# Pick a solid checkpoint from the previous notebook's CRR experiment:
#checkpoint = "results/CRR/CRR_Pendulum-v1_93a30_00000_0_2022-07-26_23-26-14/checkpoint_000028/checkpoint-28"
checkpoint = "offline_rl_data/pendulum_checkpoint/checkpoint-2"

ServeModel.deploy(config, checkpoint)
    
# That's it: Deployment created!

(ServeController pid=21320) INFO 2022-08-15 16:20:23,136 controller 21320 deployment_state.py:1277 - Adding 1 replicas to deployment 'ServeModel'.
(ServeReplica:ServeModel pid=19748) 2022-08-15 16:20:27,716	WARNING deprecation.py:47 -- DeprecationWarning: `min_iter_time_s` has been deprecated. Use `min_time_s_per_iteration` instead. This will raise an error in the future!
(ServeReplica:ServeModel pid=19748) 2022-08-15 16:20:27,717	INFO algorithm.py:351 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(ServeReplica:ServeModel pid=19748) 2022-08-15 16:20:32,499	WARNING util.py:65 -- Install gputil for GPU system monitoring.
(ServeReplica:ServeModel pid=19748) 2022-08-15 16:20:32,512	INFO trainable.py:668 -- Restored on 127.0.0.1 from checkpoint: offline_rl_data\pendulum_checkpoint
(ServeReplica:ServeModel pid=19748) 2022-08-15 16:20:32,512	INFO trainable.py:677 -- Current state after restoring: {'_iteration': 2, '_timestep

### Using the deployment for serving actions <a class="anchor" id="ray_serve_requests"></a>

Now let's send action inference requests to the existing deployment.
We'll be using a test `Pendulum-v1` environment here, pretending we are some client that would like to query the server for good Pendulum actions.

In [5]:
# Create a environment so we can step through episodes using requested, served actions.
env = gym.make("Pendulum-v1")
# Get the initial observation.
obs = env.reset()

# Request 5 actions of an episode from served policy and step through the env using the received actions.
for _ in range(3):
    # Convert numpy array to list (needed for http transfer).
    obs = obs.tolist()

    print(f"-> Sending observation {obs}")
    resp = requests.get(
        "http://localhost:8000/serve-deployment", json={"observation": obs}
    )
    # Convert to response to JSON.
    # The received JSON should include an "action" field (see our ServeModel class for details).
    response_json = resp.json()
    print(f"<- got {response_json}")

    # Convert to numpy array.
    action = np.array(response_json["action"])

    # Perform a step with the served action.
    obs, _, _, _ = env.step(action)


-> Sending observation [0.3989185690879822, 0.916986346244812, 0.12774574756622314]
<- got {'action': [-2.0]}
-> Sending observation [0.375154048204422, 0.9269624948501587, 0.5154855251312256]


(HTTPProxyActor pid=20076) INFO 2022-08-15 16:24:49,505 http_proxy 127.0.0.1 http_proxy.py:315 - GET /serve-deployment 200 3.0ms
(ServeReplica:ServeModel pid=19748) INFO 2022-08-15 16:24:49,504 ServeModel ServeModel#rCzhcV replica.py:505 - HANDLE __call__ OK 1.0ms


<- got {'action': [-2.0]}
-> Sending observation [0.3325701653957367, 0.9430785179138184, 0.9107073545455933]


(HTTPProxyActor pid=20076) INFO 2022-08-15 16:24:51,553 http_proxy 127.0.0.1 http_proxy.py:315 - GET /serve-deployment 200 4.0ms
(ServeReplica:ServeModel pid=19748) INFO 2022-08-15 16:24:51,552 ServeModel ServeModel#rCzhcV replica.py:505 - HANDLE __call__ OK 1.0ms


<- got {'action': [-2.0]}


(HTTPProxyActor pid=20076) INFO 2022-08-15 16:24:53,615 http_proxy 127.0.0.1 http_proxy.py:315 - GET /serve-deployment 200 14.0ms
(ServeReplica:ServeModel pid=19748) INFO 2022-08-15 16:24:53,611 ServeModel ServeModel#rCzhcV replica.py:505 - HANDLE __call__ OK 5.0ms
(ServeController pid=21320) 2022-08-15 16:25:06,965	INFO (unknown file):0 -- Task failed with unretryable exception: TaskID(db49530ba4f7565509a51e4fdaa6e1d25118a2b201000000).
(ServeController pid=21320) Traceback (most recent call last):
(ServeController pid=21320)   File "python\ray\_raylet.pyx", line 709, in ray._raylet.execute_task
(ServeController pid=21320)   File "python\ray\_raylet.pyx", line 713, in ray._raylet.execute_task
(ServeController pid=21320)   File "python\ray\_raylet.pyx", line 655, in ray._raylet.execute_task.function_executor
(ServeController pid=21320)   File "python\ray\_raylet.pyx", line 2243, in ray._raylet.CoreWorker.run_async_func_in_event_loop
(ServeController pid=21320)   File "C:\Programs\Anacon

### Summary

In this notebook, we have learnt:

* How to create a Ray Serve "deployment" using a custom `@serve.deployment`-tagged class
* How to query actions from this custom deployment

### Exercise 05

#### a) Run a complete episode using our Policy deployment

Use the code in the cell above and now run a complete episode through a Pendulum environment, then report the episode's rewards at the end.

In [ ]:
# Create a new environment using gym.make():
# ...

# Get the initial observation via the env's `reset()` method.
# ...

total_reward = 0.0

# Loop through an entire episode using a while loop.
# while True:

    # Remember to convert all np-arrays to lists prior to sending them via the Convert numpy array to list (needed for http transfer).
    # obs = obs.tolist()

    # Send the action request using `resp = request.get([address], json={"observation": [current observation]})`.
    # ...

    # Convert response to JSON and extract the "action" field, then convert the action to a numpy array.
    # ...

    # Perform an env step with the served action (using the env's `step([some action])` method).
    # obs, reward, done, info = env.step(...)
    
    # Add up reward.
    
    # Check done flag; if True -> break out of while loop.

    
print(f"Played one episode entirely using served actions; Total episode reward is {total_reward}.")

#### b) What would happen, if we had a "stateful" policy?

Think about the problem of having trained a neural network that uses one or more memory-capable (stateful) layers, like an LSTM layer.
Given the problem of distributed deployment (across several endpoints accessible through different addresses/ports), what do you think would be the best solution for keeping or passing around the layer's state (e.g. the LSTM's internal c- and h- states)?

* Should the server side (Ray Serve deployment) keep these tensors in between action requests?
* Or should the clients handle these states and pass them back via each new requests?

 ## References
 * [Ray Serve: Scalable and Programmable Serving](https://docs.ray.io/en/latest/serve/index.html)

➡️ [Link to previous notebook](./ex_04_offline_rl_with_rllib.ipynb) <br>
➡️ [Link to next notebook](./ex_06_rllib_end_to_end_demo.ipynb) <br>

📖 [Back to Table of Contents](./ex_00_rllib_notebooks_table_of_contents.ipynb)